Testing the pytorch  implementation of GraphCast by NVIDIA PhysicsNemo (https://github.com/NVIDIA/physicsnemo/blob/main/physicsnemo/models/graphcast/graph_cast_net.py#L568)

In [2]:
from physicsnemo.models.graphcast import GraphCastNet

In [4]:
# Test that GraphCastNet can be instantiated
import torch

print("GraphCastNet imported successfully!")
print(f"PyTorch version: {torch.__version__}")

# You can now proceed with GraphCast experiments
# Example usage (commented out until you define your specific parameters):
# model = GraphCastNet(...)
# print("GraphCastNet model created successfully!")

GraphCastNet imported successfully!
PyTorch version: 2.4.0+cu124
